<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [4]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
#!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

## 1. Scrapping data using beautiful soup and pandas

1. Scrapping data using beautiful soup and pandas
In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the boroughs and the neighborhoods that exist in each borough The latitude and logitude coordinates of each neighborhood are not present and are needed to be added in the dataset acoordingly.

Luckily, this dataset exists for free on the web on the wikipedia page. here is the link to the dataset: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.

In [5]:
#!pip install requests beautifulsoup4
#!pip install lxml
#!pip install geocoder

In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

In [8]:
postal_table = soup.find("table", attrs={"class": "wikitable sortable"})
postal_row = postal_table.tbody.find_all("tr")

l = []
for tr in postal_row:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df1 = pd.DataFrame(l, columns=["postal code", "borough", "Neighborhood"])
df1.drop(0,inplace=True)
df1.head()

,postal code,borough,Neighborhood
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
5,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [18]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
df = dfs[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [19]:
#cleaning the data
#df= df[df.Borough != 'Not assigned'] #can also be used as an alternative
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.sort_values(by=['Postal Code'], inplace=True)
df.reset_index(drop =True,inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [20]:
df.reset_index(drop =True,inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df.shape

(103, 3)

### 2. Get the latitude and longitude values of a place using csv file

In [ ]:
#code for using geocoder but it took infinte time to execute
import geocoder # import geocoder

for index in range(len(df)):
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(df['Postal Code'].iloc[index]))
      lat_lng_coords = g.latlng

    df['latitude'].iloc['index'] = lat_lng_coords[0]
    df['longitude'].iloc['index'] = lat_lng_coords[1]

In [14]:
url = 'https://cocl.us/Geospatial_data'
df_test = pd.read_csv(url)
df_test.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
df['Latitude']=df_test['Latitude']
df['Longitude']=df_test['Longitude']
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
